In [1]:
import pandas as pd
from astropy import io,table
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplt

In [2]:
plt.style.use(["default"])
plt.rcParams['figure.figsize'] = 8.0,8.0
font = {"size"   :18}
plt.rc("font",**font)

In [3]:
allstar = table.Table(io.fits.open('GALAH_DR3_main_allstar_v2.fits', mode='update')[1].data)
alls = allstar.to_pandas()

In [4]:
vac_gaia = table.Table(io.fits.open("GALAH_DR3_VAC_GaiaEDR3_v2.fits",mode="update")[1].data)
gaia = vac_gaia.to_pandas()

In [5]:
fits_din = table.Table(io.fits.open("GALAH_DR3_VAC_dynamics_v2.fits",mode="update")[1].data)
var_din = fits_din.to_pandas()

In [6]:
#Subset for only columns needed for Hertzsprung-Russell diagram and chemical abundances 
hr = alls[["sobject_id","teff","e_teff","logg","e_logg","Al_fe","fe_h","Mg_fe"]]

In [ ]:
gaia["ph"]

In [7]:
#Subset for only Coordinates and colors properties in Gaia
dyn1 = gaia[["sobject_id","ra","ra_error","dec","dec_error","parallax",
            "parallax_error","pmra","pmra_error","pmdec","pmdec_error","bp_rp","bp_g",
           "g_rp","dr2_radial_velocity","dr2_radial_velocity_error","phot_bp_mean_mag",
            "phot_g_mean_mag","phot_rp_mean_mag"]]

In [8]:
#Subset for only coordinates and velocities that will going to be useful
din1 = var_din[["sobject_id","R_Rzphi","phi_Rzphi","z_Rzphi","vR_Rzphi",
                "vT_Rzphi","vz_Rzphi","J_R","L_Z","J_Z","Energy"]]

In [9]:
merge1 = hr.merge(dyn1,how="inner",on="sobject_id")
merge2 = merge1.merge(din1,how="inner",on="sobject_id")

In [10]:
clean1 = merge2.loc[merge2["vR_Rzphi"].notna()]
clean2 = clean1.loc[(clean1["teff"].notna())&(clean1["logg"].notna())]


In [11]:
clean2["par/error"] = clean2["parallax"]/clean2["parallax_error"]
clean2 = clean2.loc[clean2["par/error"] >= 5]

/tmp/ipykernel_2441/3648064646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean2["par/error"] = clean2["parallax"]/clean2["parallax_error"]


In [12]:
clean2["distance"] = 1/(clean2["parallax"])

/tmp/ipykernel_2441/3214442858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean2["distance"] = 1/(clean2["parallax"])


In [13]:
giants = clean2.loc[(clean2["teff"] <= 6000) & (clean2["logg"] <= 3.5)]

In [27]:
solar_vec = clean2.loc[(clean2["distance"] < 3/10)]

In [20]:
giants.to_csv("giants.csv",header=True,index=False)

In [21]:
clean2.to_csv("complete.csv",header=True,index=False)

In [28]:
solar_vec.to_csv("solarnei.csv",header=True,index=False)